In [1]:
# https://help.reverb.com/hc/en-us/articles/360044717874-CSV-upload-manage-inventory-guide
# https://help.reverb.com/hc/en-us/articles/24737374642451-Replacing-listing-images-via-CSV
# https://help.reverb.com/hc/en-us/articles/212266797-How-do-I-add-many-listings-to-Reverb-at-the-same-time
# https://www.reverb-api.com/docs/updating-listing-images
# https://www.reverb-api.com/doc

In [26]:
import pandas as pd
import requests
import json
import time
import asyncio
import aiohttp
import math
import datetime

from datetime import datetime
from typing import List, Dict

personal_access_token = "1ed6918ef5f5afab0a83dd707882aea80d47c78d6e924e08b4ffdd6bdaab61dc"

headers = {
    "Content-Type": "application/hal+json",
    "Accept": "application/hal+json",
    "Accept-Version": "3.0",
    "Authorization": f"Bearer {personal_access_token}"
}


In [17]:

url = "https://api.reverb.com/api/my/listings?page=1&per_page=50"

async def fetch_page(session: aiohttp.ClientSession, url: str, headers: dict) -> dict:
    async with session.get(url, headers=headers) as response:
        return await response.json()

async def fetch_all_listings_concurrent(base_url: str, headers: dict) -> List[Dict]:
    async with aiohttp.ClientSession() as session:
        # Get first page to determine total pages
        initial_data = await fetch_page(session, base_url, headers)
        total_items = initial_data['total']
        total_pages = math.ceil(total_items / 50)

        # Create URLs for all pages
        urls = [f"{base_url.split('?')[0]}?page={page}&per_page=50" 
               for page in range(1, total_pages + 1)]

        # Fetch all pages concurrently
        tasks = [fetch_page(session, url, headers) for url in urls]
        results = await asyncio.gather(*tasks)

        # Combine all listings
        return [item for result in results for item in result['listings']]

async def timed_fetch():
    start = time.perf_counter()
    listings = await fetch_all_listings_concurrent(url, headers)
    print(f"Total listings fetched: {len(listings)}")
    print(f"Time taken: {time.perf_counter() - start:.2f} seconds")
    return listings

listings = await timed_fetch()


Total listings fetched: 467
Time taken: 3.33 seconds


In [29]:
listings_df = pd.DataFrame(listings)

now = datetime.now()

# Format it as DD-MM-YY HH:MM:SS
formatted_date_time = now.strftime("%d-%m-%y %H%M%S")

listings_df.to_csv(f'reverb_inventory_{formatted_date_time}.csv')

In [ ]:
# URLS

url_fetch = "https://api.reverb.com/api/my/listings?per_page=1000"

url_counts = "https://api.reverb.com/api/my/counts"

url_drafts = "https://api.reverb.com/api/my/listings/drafts"

url = "https://api.reverb.com/api/my/conversations"


# Finding a Listing
https://api.reverb.com/api/my/listings?sku=[sku]&state=all  # find it by SKU
https://api.reverb.com/api/listings/[listing_id] # get listing details using the listing ID. Use GET


# Update an Existing Listing
https://api.reverb.com/api/listings/[listing_id] # same as finding but use PUT


# Publishing a Listing
# https://api.reverb.com/api/listings/[listing_id] \
# --data-raw '{
#     "publish": "true"
# }'

# # Ending a Listing
# How to transition a live listing to ended:

# cURL
# curl -XPUT 
# -H "Content-Type: application/hal+json" 
# -H "Accept: application/hal+json" 
# -H "Accept-Version: 3.0" 
# -H "Authorization: Bearer [personal_token]"
# https://api.reverb.com/api/my/listings/[listing_id]/state/end 
# --data-raw '{
# "reason":"not_sold"/"reverb_sale"
# }'


SyntaxError: unterminated string literal (detected at line 23) (4158848385.py, line 23)

In [8]:
def listings():

    # API endpoint
    url = "https://api.reverb.com/api/listings"

    # Headers

    # Request payload
    payload = {
        "make": "Fender",
        "model": "Stratocaster",
        "categories": [{
            "uuid": "get these values from /api/categories/flat"
        }],
        "condition": {
            "uuid": "get these from /api/listing_conditions"
        },
        "photos": [
            "http://i.stack.imgur.com/Sv4BC.png",
            "http://i.stack.imgur.com/foo.png"
        ],
        "videos": [{
            "link": "https://www.youtube.com/watch?v=FptlwukVz2w"
        }],
        "description": "Awesome guitar",
        "finish": "Sunburst",
        "price": {
            "amount": "5000.00",
            "currency": "USD"
        },
        "title": "my favorite fender stratocaster",
        "year": "1960s",
        "sku": "12345",
        "upc": "885978110803",
        "upc_does_not_apply": "false",
        "has_inventory": True,
        "inventory": 5,
        "offers_enabled": True,
        "handmade": True,
        "shipping": {
            "rates": [
                {
                    "rate": {
                        "amount": "10.00",
                        "currency": "USD"
                    },
                    "region_code": "US_CON"
                },
                {
                    "rate": {
                        "amount": "20.00",
                        "currency": "USD"
                    },
                    "region_code": "XX"
                }
            ],
            "local": True
        }
    }
    
    # Make the POST request
    response = requests.post(url, headers=headers, json=payload)
    
    # Check response
    if response.status_code == 200:
        print("Success!")
        print(response.json())
    else:
        print(f"Error: {response.status_code}")
        print(response.text)

In [11]:
url_fetch = "https://api.reverb.com/api/my/listings?per_page=1000"
response = requests.get(url_fetch, headers=headers)
# type(response)
data_dump = json.loads(response.text)
type(data_dump)
print(data_dump.keys())
data_dump['total']
data_dump['current_page']
data_dump['total_pages']
type(data_dump['listings'])
len(data_dump['listings'])
data_dump['_links']
data_dump


dict_keys(['total', 'current_page', 'total_pages', 'listings', '_links'])


{'total': 467,
 'current_page': 1,
 'total_pages': 10,
 'listings': [{'id': 86754512,
   'make': 'Burns',
   'model': 'Hank Marvin',
   'finish': 'White',
   'year': '1964',
   'title': 'Burns Hank Marvin 1964 - White',
   'created_at': '2025-01-23T09:10:17-05:00',
   'shop_name': 'London Vintage Guitars of Denmark Street, UK & Amsterdam, NL',
   'description': "<p><b><strong>THE APACHE</strong></b></p><p>The following &nbsp;is one of the most recognised and iconic of all Burns Models purchased from its original family ownership, details as follows:</p><p><b><strong>Burns Hank Marvin</strong></b></p><p>1964</p><p>White</p><p>Scroll Headstock</p><p>3 Rezomatik Pickups</p><p>Internally / Externally Original</p><p>Original Case</p><p>Condition 9/10</p><p>Various Original Owner Pictures</p><p><b><strong>History</strong></b></p><p>The above instrument was produced by the Burns Company as a Hank B Marvin Signature Model from 1964 - 1965 ( Later Becoming Baldwin )</p><p><b><strong>Comments</s

NameError: name 'data_dump' is not defined

In [ ]:
# https://www.reverb-api.com/docs/create-listings

# curl -XPOST 
# -H "Content-Type: application/hal+json" 
# -H "Accept: application/hal+json" 
# -H "Accept-Version: 3.0" 
# -H "Authorization: Bearer [personal_token]"
# https://api.reverb.com/api/listings 
# -d '{
# 	"make": "Fender",
# 	"model": "Stratocaster",
# 	"categories": [{
# 		"uuid": "get these values from /api/categories/flat"
# 	}],
# 	"condition": {
# 		"uuid": "get these from /api/listing_conditions"
# 	},
# 	"photos": [
# 		"http://i.stack.imgur.com/Sv4BC.png",
# 		"http://i.stack.imgur.com/foo.png"
# 	],
#    "videos": [{
#             "link": "https://www.youtube.com/watch?v=FptlwukVz2w"
#   }],
# 	"description": "Awesome guitar",
# 	"finish": "Sunburst",
# 	"price": {
# 		"amount": "5000.00",
# 		"currency": "USD"
# 	},
# 	"title": "my favorite fender stratocaster",
# 	"year": "1960s",
# 	"sku": "12345",
#   "upc": "885978110803", // this string value can also be an EAN
# 	"upc_does_not_apply": "false", // use '\''true'\'' in cases where the item does not come with a UPC
# 	"has_inventory": true,
# 	"inventory": 5,
#   "offers_enabled": true, // can be either true or false
# 	"handmade": true,
# 	"shipping_profile_id": // enter your shipping profile here instead of shipping rates if applicable
# 	"shipping": { // only use this if not using shipping profiles
# 		"rates": [{
# 				"rate": {
# 					"amount": "10.00",
# 					"currency": "USD"
# 				},
# 				"region_code": "US_CON" // Continental US
# 			},
# 			{
# 				"rate": {
# 					"amount": "20.00",
# 					"currency": "USD"
# 				},
# 				"region_code": "XX" // Everywhere Else
# 			}
# 		],
# 		"local": true
# 	}
# }'

SyntaxError: unterminated string literal (detected at line 7) (3163228450.py, line 7)

In [97]:
url = "https://api.reverb.com/api/categories/flat"
url = "https://api.reverb.com//api/listing_conditions"

response = requests.get(url, headers=headers)
# type(response)
data_dump = json.loads(response.text)
data_dump

for cond in data_dump['conditions']:
    print(cond['uuid'])

7c3f45de-2ae0-4c81-8400-fdb6b1d74890
ac5b9c1e-dc78-466d-b0b3-7cf712967a48
df268ad1-c462-4ba6-b6db-e007e23922ea
ae4d9114-1bd7-4ec5-a4ba-6653af5ac84d
f7a3f48c-972a-44c6-b01a-0cd27488d3f6
98777886-76d0-44c8-865e-bb40e669e934
6a9dfcad-600b-46c8-9e08-ce6e5057921e
fbf35668-96a0-4baa-bcde-ab18d6b1b329
